In [3]:
from db_connection import get_client
from utils import *

In [4]:
printer = get_printer()
client = get_client()

question_collection = client.random_db.random_collectio

Connection to MongoDB is succesful!


In [5]:
fuzzy_match_pipeline = [
{
	"$search": {
  "index": "language_search",
	"text": {
    "query": "technoly",
		"path": "category",
		"fuzzy": {}  # Fuzzy search for flexible search - (optional)
      }
		}
	}
]

aggregate(question_collection, fuzzy_match_pipeline, printer)

[{'_id': ObjectId('67813c00cf41db2897d1e1c5'),
  'answer': 'Python',
  'category': 'Technology',
  'question': 'What programming language is known for its snake logo?',
  'value': 100},
 {'_id': ObjectId('67813b6ecf41db2897d1e18b'),
  'answer': 'Central Processing Unit',
  'category': 'Technology',
  'question': 'What does CPU stand for?',
  'value': 100},
 {'_id': ObjectId('67813b6ecf41db2897d1e196'),
  'answer': 'Hypertext Transfer Protocol',
  'category': 'Technology',
  'question': 'What does HTTP stand for?',
  'value': 200}]


In [15]:
synonym_pipeline = [
{
	"$search": {
		"index": "language_search",
    "text": {
      "query": "Python",
			"path": "category",
			"synonyms": "synonyms_mapping"
      }
		}
	},
  	{
    	"$limit": 3
		}
]


aggregate(question_collection, synonym_pipeline, printer)

[{'_id': ObjectId('67813b6ecf41db2897d1e186'),
  'answer': 'append()',
  'category': 'Python',
  'question': 'What is the method in Python to add an item to a list?',
  'value': 200},
 {'_id': ObjectId('67813b6ecf41db2897d1e183'),
  'answer': 'A NoSQL database that uses JSON-like documents.',
  'category': 'Databases',
  'question': 'What is MongoDB?',
  'value': 200},
 {'_id': ObjectId('67813b6ecf41db2897d1e185'),
  'answer': 'Structured Query Language',
  'category': 'Databases',
  'question': 'What does SQL stand for?',
  'value': 100}]


In [17]:
autocomplete_pipeline = [
{
	"$search": {
    "index": "language_search",
    "autocomplete": {
      "query": "What does ",
      "path": "question",
      "tokenOrder": "sequential",
      "fuzzy": {}
      }
		}
	},
  {
	  "$project": {
      "_id": 0,
			"question": 1
    }
	}
]

aggregate(question_collection, synonym_pipeline, printer)

[{'_id': ObjectId('67813b6ecf41db2897d1e186'),
  'answer': 'append()',
  'category': 'Python',
  'question': 'What is the method in Python to add an item to a list?',
  'value': 200},
 {'_id': ObjectId('67813b6ecf41db2897d1e183'),
  'answer': 'A NoSQL database that uses JSON-like documents.',
  'category': 'Databases',
  'question': 'What is MongoDB?',
  'value': 200},
 {'_id': ObjectId('67813b6ecf41db2897d1e185'),
  'answer': 'Structured Query Language',
  'category': 'Databases',
  'question': 'What does SQL stand for?',
  'value': 100}]


In [19]:
compound_query = [
  {
  "$search": {
    "index": "language_search",
    "compound": {
    "must": [
      {
        "text": {
          "query": ["Geography", "Physics", "Biology", "Chemistry"],
          "path": "category"
          }
      }
						],
    "mustNot": [
      {
        "text": {
          "query": ["Python", "Databases"],
          "path": "category"
          }
      }
    ],
    "should": [
      {
        "text": {
          "query": "Technology",
          "path": "answer"
          }
        }
      ]
    }
			}
				},
  {
  "$project": {
    "question": 1,
    "answer": 1,
    "category": 1,
    "score": { "$meta": "searchScore" }
    }
  },
  {
    "$limit": 3
  }
]

aggregate(question_collection, compound_query, printer)

[{'_id': ObjectId('67813c00cf41db2897d1e1ca'),
  'answer': '32°F',
  'category': 'Physics',
  'question': 'What is the freezing point of water in Fahrenheit?',
  'score': 1.2276027202606201},
 {'_id': ObjectId('67813b6ecf41db2897d1e194'),
  'answer': 'Albert Einstein',
  'category': 'Physics',
  'question': 'Who developed the theory of relativity?',
  'score': 1.2276027202606201},
 {'_id': ObjectId('67813b6ecf41db2897d1e18d'),
  'answer': 'Approximately 299,792 kilometers per second.',
  'category': 'Physics',
  'question': 'What is the speed of light?',
  'score': 1.2276027202606201}]


In [21]:
relevence_pipeline =[
{
	"$search": {
		"index": "language_search",
		"compound": {
			"must": [
				{
					"text": {
						"query": "Geography",
						"path": "category"
					}
				}
			],
			"should": [
				{
					"text": {
						"query": "capital",
						"path": "question",
						"score": { "boost": { "value": 3.0 } }
						}
				}
					]
						}
							}
},
{
	"$project": {
		"question": 1,
		"answer": 1,
		"category": 1,
		"round": 1,
		"score": { "$meta": "searchScore" }
		}
	},
{
  "$limit":3
}
]

aggregate(question_collection, relevence_pipeline, printer)

[{'_id': ObjectId('67813b6ecf41db2897d1e187'),
  'answer': 'Paris',
  'category': 'Geography',
  'question': 'What is the capital of France?',
  'score': 0.8210218548774719},
 {'_id': ObjectId('67813c00cf41db2897d1e1bd'),
  'answer': 'Tokyo',
  'category': 'Geography',
  'question': 'What is the capital of Japan?',
  'score': 0.8210218548774719},
 {'_id': ObjectId('67813c00cf41db2897d1e1c6'),
  'answer': 'The Nile River',
  'category': 'Geography',
  'question': 'What is the longest river in the world?',
  'score': 0.8210218548774719}]
